In [ ]:
import numpy as np
import cv2
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
import seaborn as sns
import warnings
import pickle
import os
from datetime import datetime
import json
import hashlib
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from scipy import ndimage

warnings.filterwarnings('ignore')
print("✅ Librerías cargadas")

# ============================================
# CONFIGURACIÓN
# ============================================
DATA_PATH = 'color/color-img'        # Ruta a tu dataset
OUTPUT_PATH = 'processed_data'       # Donde guardar datos procesados

TARGET_SAMPLES_PER_CLASS = 1500  # MODIFICADO: 1500 imágenes por clase
EPOCHS = 200
BATCH_SIZE = 8
TRAIN_TEST_SPLIT = 0.8  # MODIFICADO: 80-20 split
IMG_SIZE = (256, 256)

print(f"📊 Configuración: {TARGET_SAMPLES_PER_CLASS} imgs/clase, {IMG_SIZE} size, {TRAIN_TEST_SPLIT:.0%} train split")
os.makedirs(OUTPUT_PATH, exist_ok=True)

CORN_CLASSES = {
    0: 'Tizón foliar del norte (Northern Leaf Blight)',
    1: 'Mancha gris de la hoja (Cercospora/Gray Leaf Spot)', 
    2: 'Roya común (Common Rust)',
    3: 'Sano (Healthy)'
}

CORN_CLASSES_SHORT = {
    0: 'Northern LB',
    1: 'Gray LS', 
    2: 'Common Rust',
    3: 'Healthy'
}

CORN_FOLDERS = [
    'Corn_(maize)___Northern_Leaf_Blight',
    'Corn_(maize)___Cercospora_leaf_spot',
    'Corn_(maize)___Common_rust_',
    'Corn_(maize)___healthy'
]

PREPROCESSING_CONFIG = {
    'clahe_clip_limit': 2.0,
    'clahe_tile_grid_size': (8, 8),
    'amf_min_window': 3,
    'amf_max_window': 7,
}

# Configuración de augmentation para alcanzar 1500 imágenes por clase
AUGMENTATION_CONFIG = {
    'rotation_range': 45,
    'zoom_range': 0.2,
    'horizontal_flip': True,
    'vertical_flip': False,
    'brightness_range': [1.1, 1.5],
    'channel_shift_range': 0.1,
    'width_shift_range': 0.2,
    'height_shift_range': 0.2,
    'shear_range': 0.2,
    'fill_mode': 'nearest'
}

# ============================================
# FUNCIONES DE EXPLORACIÓN DEL DATASET
# ============================================
def get_all_image_files(folder_path):
    """Obtener todos los archivos de imagen"""
    all_files = []
    base_extensions = ['*.jpg', '*.jpeg', '*.png']
    
    for ext in base_extensions:
        files_lower = list(folder_path.glob(ext))
        files_upper = list(folder_path.glob(ext.upper()))
        all_files.extend(files_lower)
        all_files.extend(files_upper)
    
    unique_files = list(set(str(f.resolve()) for f in all_files))
    return [Path(f) for f in unique_files]

def explore_dataset(data_path):
    """Explorar la estructura del dataset antes de cargar las imágenes"""
    print("📂 Explorando dataset...")
    
    data_path = Path(data_path)
    
    if not data_path.exists():
        print(f"❌ Error: La ruta {data_path} no existe")
        return False, {}
    
    total_images = 0
    class_distribution = {}
    class_info = {}
    
    for idx, folder in enumerate(CORN_FOLDERS):
        folder_path = data_path / folder
        
        if not folder_path.exists():
            print(f"❌ {CORN_CLASSES_SHORT[idx]}: Carpeta no encontrada")
            class_distribution[idx] = 0
            class_info[idx] = {'path': str(folder_path), 'files': [], 'count': 0}
            continue
        
        all_files = get_all_image_files(folder_path)
        
        class_distribution[idx] = len(all_files)
        class_info[idx] = {
            'path': str(folder_path),
            'files': [str(f) for f in all_files],
            'count': len(all_files)
        }
        total_images += len(all_files)
        
        print(f"✅ {CORN_CLASSES_SHORT[idx]}: {len(all_files)} imágenes")
    
    if total_images > 0:
        print(f"\n📊 Total: {total_images} imágenes")
        print(f"🎯 Se generarán: {TARGET_SAMPLES_PER_CLASS * 4} imágenes ({TARGET_SAMPLES_PER_CLASS} por clase)")
    
    return total_images > 0, class_info

# ============================================
# PREPROCESAMIENTO HSV PARA ENFERMEDADES (NUEVO)
# ============================================
def apply_hsv_enhancement_for_disease(image):
    """
    Aplicar mejora HSV solo para detectar mejor enfermedades
    NO se aplica a imágenes de clase Healthy
    """
    # Convertir a HSV
    hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    h, s, v = cv2.split(hsv)
    
    # Mejorar saturación para resaltar colores de enfermedades
    # Las enfermedades suelen tener colores amarillentos, marrones o rojizos
    s_enhanced = cv2.multiply(s, 1.1)  # Aumentar saturación 30%
    s_enhanced = np.clip(s_enhanced, 0, 255).astype(np.uint8)
    
    # Ajustar value para mejorar contraste
    clahe_v = cv2.createCLAHE(clipLimit=1.5, tileGridSize=(6,6))
    v_enhanced = clahe_v.apply(v)
    
    # Recombinar canales
    hsv_enhanced = cv2.merge([h, s_enhanced, v_enhanced])
    
    # Convertir de vuelta a RGB
    rgb_enhanced = cv2.cvtColor(hsv_enhanced, cv2.COLOR_HSV2RGB)
    
    return rgb_enhanced

# ============================================
# ADAPTIVE MEDIAN FILTER OPTIMIZADO
# ============================================
def adaptive_median_filter_fast(image, min_window=3, max_window=7):
    """
    Versión optimizada del AMF usando operaciones vectorizadas
    """
    result = np.copy(image).astype(np.float64)
    rows, cols = image.shape
    processed = np.zeros((rows, cols), dtype=bool)
    
    for window_size in range(min_window, max_window + 1, 2):
        if np.all(processed):
            break
            
        median_filtered = ndimage.median_filter(image, size=window_size)
        min_filtered = ndimage.minimum_filter(image, size=window_size)
        max_filtered = ndimage.maximum_filter(image, size=window_size)
        
        level_A_valid = (min_filtered < median_filtered) & (median_filtered < max_filtered)
        level_B_valid = (min_filtered < image) & (image < max_filtered)
        
        keep_original = level_A_valid & level_B_valid & ~processed
        result[keep_original] = image[keep_original]
        processed[keep_original] = True
        
        use_median = level_A_valid & ~level_B_valid & ~processed
        result[use_median] = median_filtered[use_median]
        processed[use_median] = True
    
    if not np.all(processed):
        final_median = ndimage.median_filter(image, size=max_window)
        result[~processed] = final_median[~processed]
    
    return result.astype(np.uint8)

# ============================================
# PIPELINE DE PREPROCESAMIENTO COMPLETO
# ============================================
def preprocess_image_pipeline(image_path, class_idx, visualize=False):
    """
    Pipeline de preprocesamiento del paper + HSV para enfermedades:
    RGB → HSV enhancement (si es enfermedad) → L*a*b* → CLAHE → AMF → Resize → Normalize
    
    Args:
        image_path: Ruta de la imagen o array numpy
        class_idx: Índice de la clase (0-2: enfermedades, 3: healthy)
        visualize: Si mostrar el proceso paso a paso
    
    Returns:
        Imagen preprocesada normalizada (0-1) con shape (H, W, 3)
    """
    # Leer imagen
    if isinstance(image_path, str):
        img = cv2.imread(image_path)
        if img is None:
            return None
        original = img.copy()
    else:
        img = image_path.copy()
        original = img.copy()
    
    try:
        # Convertir BGR a RGB
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # NUEVO: Aplicar HSV enhancement 
        if class_idx == 1:  # solo gray ls
            img_rgb = apply_hsv_enhancement_for_disease(img_rgb)
        
        # Pipeline completo del paper
        # 1. Convertir RGB a LAB
        lab = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2LAB)
        l_channel, a_channel, b_channel = cv2.split(lab)
        
        # 2. Aplicar CLAHE al canal L
        clahe = cv2.createCLAHE(
            clipLimit=PREPROCESSING_CONFIG['clahe_clip_limit'],
            tileGridSize=PREPROCESSING_CONFIG['clahe_tile_grid_size']
        )
        l_clahe = clahe.apply(l_channel)
        
        # 3. Aplicar AMF al canal L con CLAHE
        l_amf = adaptive_median_filter_fast(
            l_clahe, 
            min_window=PREPROCESSING_CONFIG['amf_min_window'], 
            max_window=PREPROCESSING_CONFIG['amf_max_window']
        )
        
        # 4. Recombinar canales
        lab_amf = cv2.merge([l_amf, a_channel, b_channel])
        
        # 5. Convertir de vuelta a RGB
        img_filtered = cv2.cvtColor(lab_amf, cv2.COLOR_LAB2RGB)
        
        # 6. Resize
        img_resized = cv2.resize(img_filtered, IMG_SIZE)
        
        # 7. Normalizar
        img_normalized = img_resized / 255.0
        
        # Verificar formato final
        if len(img_normalized.shape) == 2:
            img_normalized = np.stack([img_normalized] * 3, axis=-1)
        elif img_normalized.shape[-1] != 3:
            return None
        
        # Verificación final
        assert img_normalized.shape == (IMG_SIZE[0], IMG_SIZE[1], 3)
        assert 0 <= img_normalized.min() <= img_normalized.max() <= 1
        
        return img_normalized
        
    except Exception as e:
        print(f"⚠️ Error en preprocesamiento: {str(e)}")
        return None

# ============================================
# GENERACIÓN DE IMÁGENES SINTÉTICAS
# ============================================
def generate_synthetic_images(images, labels, target_per_class=1500):
    """
    Genera imágenes sintéticas usando augmentation para alcanzar target_per_class
    """
    print(f"\n🔄 Generando imágenes sintéticas para alcanzar {target_per_class} por clase...")
    
    augmented_images = []
    augmented_labels = []
    
    # Crear generador de augmentation
    datagen = ImageDataGenerator(**AUGMENTATION_CONFIG)
    
    for class_idx in range(4):
        class_images = images[labels == class_idx]
        current_count = len(class_images)
        needed = target_per_class - current_count
        
        print(f"\n{CORN_CLASSES_SHORT[class_idx]}:")
        print(f"  • Originales: {current_count}")
        print(f"  • Necesarias: {needed}")
        
        # Agregar todas las imágenes originales
        augmented_images.extend(class_images)
        augmented_labels.extend([class_idx] * current_count)
        
        if needed > 0:
            # Generar imágenes sintéticas
            generated_count = 0
            iterations = 0
            max_iterations = needed * 5  # Prevenir bucle infinito
            
            while generated_count < needed and iterations < max_iterations:
                # Seleccionar imagen aleatoria de la clase
                idx = np.random.randint(0, current_count)
                img = class_images[idx:idx+1]
                
                # Generar una versión aumentada
                for batch in datagen.flow(img, batch_size=1):
                    augmented_img = batch[0]
                    augmented_images.append(augmented_img)
                    augmented_labels.append(class_idx)
                    generated_count += 1
                    iterations += 1
                    break
                
                if generated_count % 100 == 0 and generated_count > 0:
                    print(f"    Generadas: {generated_count}/{needed}")
            
            print(f"  ✅ Total generadas: {generated_count}")
        else:
            print(f"  ✅ No se necesitan imágenes adicionales")
    
    # Convertir a arrays numpy
    augmented_images = np.array(augmented_images, dtype=np.float32)
    augmented_labels = np.array(augmented_labels, dtype=np.int32)
    
    # Mezclar los datos
    indices = np.random.permutation(len(augmented_images))
    augmented_images = augmented_images[indices]
    augmented_labels = augmented_labels[indices]
    
    print(f"\n📊 Total final: {len(augmented_images)} imágenes")
    
    return augmented_images, augmented_labels

# ============================================
# CARGA Y PROCESAMIENTO DEL DATASET
# ============================================
def load_and_preprocess_dataset(data_path, show_progress=True):
    """
    Cargar y preprocesar imágenes del dataset aplicando CLAHE+AMF+HSV
    """
    images = []
    labels = []
    file_paths = []
    processing_log = []
    
    print("🔧 Cargando y preprocesando con CLAHE + AMF + HSV enhancement...")
    
    total_loaded = 0
    total_errors = 0
    
    for idx, folder in enumerate(CORN_FOLDERS):
        folder_path = Path(data_path) / folder
        class_name = CORN_CLASSES_SHORT[idx]
        
        print(f"\n{class_name}:")
        
        all_imgs = get_all_image_files(folder_path)
        
        if not all_imgs:
            print(f"  ⚠️ No se encontraron imágenes")
            continue
        
        print(f"  Disponibles: {len(all_imgs)} imágenes")
        
        class_loaded = 0
        class_errors = 0
        
        for img_path in all_imgs:
            try:
                # Preprocesar imagen con índice de clase para HSV enhancement
                processed_img = preprocess_image_pipeline(str(img_path), idx, visualize=False)
                
                if processed_img is not None:
                    images.append(processed_img)
                    labels.append(idx)
                    file_paths.append(str(img_path))
                    class_loaded += 1
                    total_loaded += 1
                    
                    if show_progress and total_loaded % 100 == 0:
                        print(f"    Procesadas: {total_loaded}")
                        
                else:
                    class_errors += 1
                    total_errors += 1
                    
            except Exception as e:
                print(f"    ⚠️ Error en {img_path.name}: {str(e)}")
                class_errors += 1
                total_errors += 1
        
        print(f"  ✅ Cargadas: {class_loaded}, Errores: {class_errors}")
        
        processing_log.append({
            'class': class_name,
            'loaded': class_loaded,
            'errors': class_errors,
            'success_rate': class_loaded / (class_loaded + class_errors) if (class_loaded + class_errors) > 0 else 0
        })
    
    if len(images) == 0:
        raise ValueError("No se pudieron cargar imágenes válidas")
    
    # Convertir a arrays numpy
    X = np.array(images, dtype=np.float32)
    y = np.array(labels, dtype=np.int32)
    
    print(f"\n✅ DATASET CARGADO:")
    print(f"  📊 Total imágenes: {len(X)}")
    print(f"  📐 Shape: {X.shape}")
    print(f"  🎯 Clases: {len(np.unique(y))}")
    print(f"  ❌ Errores: {total_errors}")
    
    # Mostrar distribución por clase
    print(f"\n📊 DISTRIBUCIÓN ORIGINAL POR CLASE:")
    for idx, class_name in CORN_CLASSES_SHORT.items():
        count = np.sum(y == idx)
        percentage = (count / len(y)) * 100 if len(y) > 0 else 0
        print(f"  • {class_name}: {count} ({percentage:.1f}%)")
    
    # Guardar log de procesamiento
    log_df = pd.DataFrame(processing_log)
    log_df.to_csv(Path(OUTPUT_PATH) / 'preprocessing_log.csv', index=False)
    
    return X, y, file_paths, processing_log

# ============================================
# PIPELINE PRINCIPAL
# ============================================
def complete_preprocessing_pipeline():
    """
    Pipeline completo con generación de imágenes sintéticas y división 80-20
    """
    print("=" * 60)
    print("PIPELINE DE PREPROCESAMIENTO v3")
    print("CLAHE + AMF + HSV Enhancement + Synthetic Generation")
    print("=" * 60)
    
    # Paso 1: Explorar dataset
    print("\nPASO 1: Exploración del dataset")
    dataset_ok, dataset_info = explore_dataset(DATA_PATH)
    
    if not dataset_ok:
        print("Error: Dataset no válido")
        return None
    
    # Paso 2: Cargar y preprocesar imágenes originales
    print("\nPASO 2: Carga y preprocesamiento")
    X_original, y_original, file_paths, processing_log = load_and_preprocess_dataset(
        DATA_PATH, show_progress=True
    )
    
    # Paso 3: Generar imágenes sintéticas para alcanzar 1500 por clase
    print("\nPASO 3: Generación de imágenes sintéticas")
    X_augmented, y_augmented = generate_synthetic_images(
        X_original, y_original, target_per_class=TARGET_SAMPLES_PER_CLASS
    )
    
    # Paso 4: División estratificada 80-20
    print("\nPASO 4: División estratificada 80-20")
    X_train, X_test, y_train, y_test = train_test_split(
        X_augmented, y_augmented, 
        test_size=(1 - TRAIN_TEST_SPLIT),
        random_state=42,
        stratify=y_augmented
    )
    
    # Convertir a one-hot
    y_train_cat = to_categorical(y_train, num_classes=4)
    y_test_cat = to_categorical(y_test, num_classes=4)
    
    print(f"\nDivisión final:")
    print(f"  • Entrenamiento: {len(X_train)} imágenes")
    print(f"  • Test: {len(X_test)} imágenes")
    
    # Mostrar distribución
    print("\nDISTRIBUCIÓN POR CLASE:")
    for split_name, y_data in [("Train", y_train), ("Test", y_test)]:
        print(f"\n{split_name}:")
        unique, counts = np.unique(y_data, return_counts=True)
        for cls, count in zip(unique, counts):
            percentage = (count / len(y_data)) * 100
            print(f"  • {CORN_CLASSES_SHORT[cls]}: {count} ({percentage:.1f}%)")
    
    # Preparar datos para guardar
    processed_data = {
        'X_train': X_train,
        'X_test': X_test,
        'y_train': y_train_cat,
        'y_test': y_test_cat,
        'y_train_raw': y_train,
        'y_test_raw': y_test,
        'class_names': CORN_CLASSES_SHORT,
        'num_classes': 4,
        'file_paths': file_paths,
        'processing_info': {
            'method': 'v3_clahe_amf_hsv_synthetic',
            'preprocessing': 'CLAHE + AMF + HSV Enhancement for diseases',
            'synthetic_generation': f'{TARGET_SAMPLES_PER_CLASS} per class',
            'original_samples': len(X_original),
            'augmented_samples': len(X_augmented),
            'train_samples': len(X_train),
            'test_samples': len(X_test),
            'train_test_split': f'{TRAIN_TEST_SPLIT:.0%}-{1-TRAIN_TEST_SPLIT:.0%}',
            'processing_date': datetime.now().isoformat(),
            'augmentation_config': AUGMENTATION_CONFIG,
            'preprocessing_config': PREPROCESSING_CONFIG
        }
    }
    
    # Guardar datos
    save_path = Path(OUTPUT_PATH)
    save_path.mkdir(exist_ok=True)
    
    # Guardar arrays numpy
    np.savez_compressed(
        save_path / 'preprocessed_data_v3.npz',
        **{k: v for k, v in processed_data.items() if isinstance(v, np.ndarray)}
    )
    
    # Guardar metadata
    with open(save_path / 'metadata_v3.pkl', 'wb') as f:
        pickle.dump({
            'class_names': CORN_CLASSES_SHORT,
            'num_classes': 4,
            'file_paths': file_paths,
            'processing_info': processed_data['processing_info'],
            'processing_log': processing_log,
            'data_shapes': {k: v.shape for k, v in processed_data.items() if isinstance(v, np.ndarray)},
            'CORN_CLASSES': CORN_CLASSES,
            'CORN_CLASSES_SHORT': CORN_CLASSES_SHORT,
            'AUGMENTATION_CONFIG': AUGMENTATION_CONFIG,
            'PREPROCESSING_CONFIG': PREPROCESSING_CONFIG
        }, f)
    
    print(f"\n✅ Datos guardados en:")
    print(f"  • {save_path}/preprocessed_data_v3.npz")
    print(f"  • {save_path}/metadata_v3.pkl")
    print("\n" + "=" * 60)
    print("PIPELINE COMPLETADO EXITOSAMENTE")
    print("=" * 60)
    
    return processed_data

# ============================================
# FUNCIONES DE VISUALIZACIÓN
# ============================================
def visualize_preprocessing_comparison():
    """
    Visualizar comparación de preprocesamiento con y sin HSV enhancement
    """
    print("\n📊 Generando comparación visual...")
    
    # Cargar una imagen de cada clase
    fig, axes = plt.subplots(4, 3, figsize=(12, 16))
    
    for idx, folder in enumerate(CORN_FOLDERS):
        folder_path = Path(DATA_PATH) / folder
        imgs = get_all_image_files(folder_path)
        
        if imgs:
            # Leer imagen original
            img = cv2.imread(str(imgs[0]))
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            
            # Original
            axes[idx, 0].imshow(img_rgb)
            axes[idx, 0].set_title(f'{CORN_CLASSES_SHORT[idx]} - Original')
            axes[idx, 0].axis('off')
            
            # Con preprocesamiento estándar
            processed_standard = preprocess_image_pipeline(str(imgs[0]), 3, visualize=False)  # Como si fuera healthy
            if processed_standard is not None:
                axes[idx, 1].imshow(processed_standard)
                axes[idx, 1].set_title('CLAHE + AMF')
                axes[idx, 1].axis('off')
            
            # Con HSV enhancement (solo para enfermedades)
            processed_hsv = preprocess_image_pipeline(str(imgs[0]), idx, visualize=False)
            if processed_hsv is not None:
                axes[idx, 2].imshow(processed_hsv)
                title = 'CLAHE + AMF + HSV' if idx < 3 else 'CLAHE + AMF (sin HSV)'
                axes[idx, 2].set_title(title)
                axes[idx, 2].axis('off')
    
    plt.suptitle('Comparación de Preprocesamiento', fontsize=16)
    plt.tight_layout()
    plt.savefig(Path(OUTPUT_PATH) / 'preprocessing_comparison.png', dpi=150, bbox_inches='tight')
    plt.show()
    print("✅ Visualización guardada en processed_data/preprocessing_comparison.png")

def verify_saved_data():
    """
    Verificar y mostrar información sobre los datos guardados
    """
    print("\n🔍 Verificando datos guardados...")
    
    # Cargar datos
    data_path = Path(OUTPUT_PATH) / 'preprocessed_data_v3.npz'
    metadata_path = Path(OUTPUT_PATH) / 'metadata_v3.pkl'
    
    if data_path.exists() and metadata_path.exists():
        data = np.load(data_path)
        with open(metadata_path, 'rb') as f:
            metadata = pickle.load(f)
        
        print("\n✅ Datos cargados correctamente:")
        print(f"  • X_train shape: {data['X_train'].shape}")
        print(f"  • X_test shape: {data['X_test'].shape}")
        print(f"  • y_train shape: {data['y_train'].shape}")
        print(f"  • y_test shape: {data['y_test'].shape}")
        
        # Mostrar algunas imágenes de entrenamiento
        fig, axes = plt.subplots(2, 4, figsize=(16, 8))
        for i in range(8):
            if i < len(data['X_train']):
                axes[i//4, i%4].imshow(data['X_train'][i])
                class_idx = np.argmax(data['y_train'][i])
                axes[i//4, i%4].set_title(metadata['class_names'][class_idx])
                axes[i//4, i%4].axis('off')
        
        plt.suptitle('Muestras del dataset procesado', fontsize=14)
        plt.tight_layout()
        plt.show()
    else:
        print("⚠️ No se encontraron datos guardados. Ejecuta el pipeline primero.")

# ============================================
# EJECUCIÓN PRINCIPAL
# ============================================
if __name__ == "__main__":
    # Ejecutar pipeline completo
    result = complete_preprocessing_pipeline()
    
    # Generar visualización comparativa
    if result is not None:
        visualize_preprocessing_comparison()
        verify_saved_data()